In [21]:
import numpy as np
import pandas as pd
from pathlib import Path
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

In [2]:
train_df = pd.read_csv(Path('Resources/Generator/2019loans.csv'))
test_df = pd.read_csv(Path('Resources/Generator/2020Q1loans.csv'))

In [36]:
train_df.head(1)

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,target
0,7000.0,0.1894,256.38,MORTGAGE,75000.0,Not Verified,n,28.62,0.0,2.0,...,87.5,0.0,0.0,352260.0,62666.0,35000.0,10000.0,N,N,low_risk


In [35]:
test_df.head(1)

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,target
0,40000.0,0.1033,856.4,RENT,128700.0,Source Verified,n,12.47,0.0,1.0,...,57.1,0.0,0.0,63915.0,49510.0,49400.0,14515.0,Y,N,low_risk


In [5]:
# Convert categorical data to numeric and separate target feature for training data
X_train = train_df.drop(columns=['target'])
y_train = train_df['target']
X_train_coded = pd.get_dummies(X_train, columns=['home_ownership', 'verification_status', 'pymnt_plan', 'initial_list_status', 'application_type', 'hardship_flag', 'debt_settlement_flag'])
y_train_coded = pd.get_dummies(y_train, columns=['target'])
y_train_coded = y_train_coded['high_risk'].ravel()

In [7]:
# Convert categorical data to numeric and separate target feature for testing data
X_test = test_df.drop(columns=['target'])
y_test = test_df['target']
X_test_coded = pd.get_dummies(X_test, columns=['home_ownership', 'verification_status', 'pymnt_plan', 'initial_list_status', 'application_type', 'hardship_flag', 'debt_settlement_flag'])
y_test_coded = pd.get_dummies(y_test, columns=['target'])
y_test_coded = y_test_coded['high_risk'].ravel()

In [17]:
# add missing dummy variables to testing set
for col in X_train_coded.columns:
    if col not in X_test_coded.columns:
        X_test_coded[col] = 0

### Predictions:
I think the random forests classifier will be more accurate. The ensemble approach will be more robust and better suited to handle complex datasets.

In [18]:
# Train the Logistic Regression model on the unscaled data and print the model score
classifier = LogisticRegression(max_iter=1000)
classifier.fit (X_train_coded, y_train_coded)
print(f"Training Data Score: {classifier.score(X_train_coded, y_train_coded)}")
print(f"Testing Data Score: {classifier.score(X_test_coded, y_test_coded)}")

Training Data Score: 0.7044334975369458
Testing Data Score: 0.5686941726924712


C:\Users\Dan\anaconda3\envs\dev\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [19]:
# Train a Random Forest Classifier model and print the model score
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)
rf_model = rf_model.fit(X_train_coded, y_train_coded)
print(f"Training Data Score: {rf_model.score(X_train_coded, y_train_coded)}")
print(f"Testing Data Score: {rf_model.score(X_test_coded, y_test_coded)}")

Training Data Score: 1.0
Testing Data Score: 0.6427052318162484


### Predictions for Scaled Data:
I think the random forests classifier will be even more accurate with scaled data for the same reasons; with better data the ensemble approach should work even better.

In [23]:
# Scale the data
scaler = StandardScaler().fit(X_train_coded)
X_train_scaled = scaler.transform(X_train_coded)
X_test_scaled = scaler.transform(X_test_coded)

In [25]:
# Train the Logistic Regression model on the scaled data and print the model score
classifier = LogisticRegression(max_iter=1000)
classifier.fit (X_train_scaled, y_train_coded)
print(f"Training Data Score: {classifier.score(X_train_scaled, y_train_coded)}")
print(f"Testing Data Score: {classifier.score(X_test_scaled, y_test_coded)}")

Training Data Score: 0.710919540229885
Testing Data Score: 0.7598894087622289


In [24]:
# Train a Random Forest Classifier model on the scaled data and print the model score
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)
rf_model = rf_model.fit(X_train_scaled, y_train_coded)
print(f"Training Data Score: {rf_model.score(X_train_scaled, y_train_coded)}")
print(f"Testing Data Score: {rf_model.score(X_test_scaled, y_test_coded)}")

Training Data Score: 1.0
Testing Data Score: 0.6439812845597618


### Results:
I was quite wrong, the logistic regression performed better. I should have considered its perfect training score which indicates overfitting. Other possible reasons are that the model's simplicity was an advantage for this dataset. It seems scaling really benefits logistic regressions while barely affecting random forest models in this case.